In [1]:
import pandas as pd

In [19]:
from sklearn.model_selection import train_test_split

In [39]:
from transformers import BertTokenizer, BertForNextSentencePrediction, Trainer, TrainingArguments

In [37]:
from datasets import metric, load_metric

In [26]:
from torch.utils.data import Dataset, DataLoader

In [12]:
class Seq2SeqDataset(Dataset):
    def __init__(self, df):
        self.df = df
    def __getitem__(self, idx):
        return self.df['source'][idx], self.df['target'][idx]
    def __len__(self):
        return len(self.df)

In [15]:
def collate(batch):
    input_ids = [x[0] for x in batch]
    labels = [x[1] for x in batch]
    
    input_ids = tokenizer(input_ids, return_tensors='pt', padding=True).input_ids
    labels = tokenizer(labels, return_tensors='pt', padding=True).input_ids
    
    return input_ids, labels

In [27]:
def load_dataset(path):
    df = pd.read_csv('data_defense/s2sdataset.csv')
    df, _ = train_test_split(df, test_size=0.6)
    train_df, test_df = train_test_split(df, test_size=0.05)
    return Seq2SeqDataset(train_df), Seq2SeqDataset(test_df)

In [30]:
params = {
    'batch_size': 8,
    'shuffle': True,
}

In [28]:
train_ds, test_ds = load_dataset('data_defense/s2sdataset.csv')

In [31]:
train_loader = DataLoader(train_ds, batch_size=params['batch_size'], shuffle=params['shuffle'], collate_fn=collate)
test_loader = DataLoader(test_ds, batch_size=params['batch_size'], shuffle=params['shuffle'], collate_fn=collate)

In [ ]:
model = BertForNextSentencePrediction.from_pretrained()

In [40]:
# Metrics
metric = load_metric("accuracy")

training_args = TrainingArguments(output_dir="models/imdb_classifier", evaluation_strategy="epoch")

def compute_metrics(eval_pred):
  logits, labels = eval_pred
  predictions = np.argmax(logits, axis=-1)
  return metric.compute(predictions=predictions, references=labels)

# Train
trainer = Trainer(
  model=model,
  args=training_args,
  train_dataset=dataset,
  eval_dataset=dataset,
  compute_metrics=compute_metrics,
)
trainer.train()

NameError: name 'model' is not defined